> # Model Mangement #

It is easy to store trained keras model if we use sequential or functional model.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings ; warnings.filterwarnings("ignore")

Use california data for example.

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


housing = fetch_california_housing()

X, y = housing.data, housing.target

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y)

X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [3]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

model.compile(loss='mean_squared_error', optimizer='sgd')

In [4]:
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

Epoch 1/30
363/363 [==============================] - 0s 1ms/step - loss: 0.8789 - val_loss: 0.5370
Epoch 2/30
363/363 [==============================] - 0s 730us/step - loss: 0.5002 - val_loss: 0.4818
Epoch 3/30
363/363 [==============================] - 0s 821us/step - loss: 0.4675 - val_loss: 0.4708
Epoch 4/30
363/363 [==============================] - 0s 760us/step - loss: 0.4642 - val_loss: 0.4705
Epoch 5/30
363/363 [==============================] - 0s 737us/step - loss: 0.4456 - val_loss: 0.4525
Epoch 6/30
363/363 [==============================] - 0s 739us/step - loss: 0.4373 - val_loss: 0.4766
Epoch 7/30
363/363 [==============================] - 0s 710us/step - loss: 0.4280 - val_loss: 0.4479
Epoch 8/30
363/363 [==============================] - 0s 709us/step - loss: 0.4219 - val_loss: 0.4399
Epoch 9/30
363/363 [==============================] - 0s 718us/step - loss: 0.4156 - val_loss: 0.4312
Epoch 10/30
363/363 [==============================] - 0s 769us/step - loss: 0.4207 

In [5]:
model.save("keras_model1.h5")

we can save a model by one code line. We are also able to load model and make predictions like above.

In [6]:
model = keras.models.load_model("keras_model1.h5")

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                270       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


### #
However, generaly train step continues for a few hours. In this case, we have to save some checkpoints during the train step not to loss all from computer error. So we use **callback**.

In [8]:
checkpoint = keras.callbacks.ModelCheckpoint("keras_model1.h5")
history = model.fit(X_train, y_train, epochs=10, callbacks=[checkpoint])

Epoch 1/10
363/363 [==============================] - 0s 750us/step - loss: 0.3531
Epoch 2/10
363/363 [==============================] - 0s 572us/step - loss: 0.3523
Epoch 3/10
363/363 [==============================] - 0s 628us/step - loss: 0.3650
Epoch 4/10
363/363 [==============================] - 0s 578us/step - loss: 0.3499
Epoch 5/10
363/363 [==============================] - 0s 605us/step - loss: 0.3513
Epoch 6/10
363/363 [==============================] - 0s 657us/step - loss: 0.3498
Epoch 7/10
363/363 [==============================] - 0s 621us/step - loss: 0.3556
Epoch 8/10
363/363 [==============================] - 0s 585us/step - loss: 0.3906
Epoch 9/10
363/363 [==============================] - 0s 616us/step - loss: 0.3624
Epoch 10/10
363/363 [==============================] - 0s 622us/step - loss: 0.3538


If we use validation set during train step, we can set **save_best_only=True** when we make model checkpoint. Then it saves model only at the best validation set score. We don't need to worry about overfitting.

In [9]:
checkpoint1 = keras.callbacks.ModelCheckpoint("keras_model1.h5", save_best_only=True)

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid), callbacks=[checkpoint1])

Epoch 1/10
363/363 [==============================] - 0s 1ms/step - loss: 0.3424 - val_loss: 0.3617
Epoch 2/10
363/363 [==============================] - 0s 834us/step - loss: 0.3427 - val_loss: 0.3656
Epoch 3/10
363/363 [==============================] - 0s 773us/step - loss: 0.3505 - val_loss: 0.3594
Epoch 4/10
363/363 [==============================] - 0s 785us/step - loss: 0.3427 - val_loss: 0.3551
Epoch 5/10
363/363 [==============================] - 0s 789us/step - loss: 0.3399 - val_loss: 0.3557
Epoch 6/10
363/363 [==============================] - 0s 768us/step - loss: 0.3353 - val_loss: 0.3564
Epoch 7/10
363/363 [==============================] - ETA: 0s - loss: 0.3410- ETA: 0s - loss: 0. - 0s 825us/step - loss: 0.3387 - val_loss: 0.3522
Epoch 8/10
363/363 [==============================] - 0s 896us/step - loss: 0.3353 - val_loss: 0.3518
Epoch 9/10
363/363 [==============================] - 0s 931us/step - loss: 0.3324 - val_loss: 0.3505
Epoch 10/10
363/363 [==================

And we can load the best model.

In [10]:
model = keras.models.load_model("keras_model1.h5")

Another method to realize earlystopping is to use **EarlyStopping** callback. It stops train if validation score doesn't imporve anymore. we can use both checkpoint save callback and earlystopping callback.

In [11]:
early_stop = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[checkpoint1, early_stop])

Epoch 1/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3295 - val_loss: 0.3568
Epoch 2/100
363/363 [==============================] - 0s 830us/step - loss: 0.3305 - val_loss: 0.3492
Epoch 3/100
363/363 [==============================] - 0s 868us/step - loss: 0.3292 - val_loss: 0.3479
Epoch 4/100
363/363 [==============================] - 0s 850us/step - loss: 0.3330 - val_loss: 0.3493
Epoch 5/100
363/363 [==============================] - 0s 827us/step - loss: 0.3287 - val_loss: 0.3531
Epoch 6/100
363/363 [==============================] - 0s 899us/step - loss: 0.3263 - val_loss: 0.4994
Epoch 7/100
363/363 [==============================] - 0s 850us/step - loss: 0.3336 - val_loss: 0.3531
Epoch 8/100
363/363 [==============================] - 0s 835us/step - loss: 0.3263 - val_loss: 0.3486
Epoch 9/100
363/363 [==============================] - 0s 928us/step - loss: 0.3262 - val_loss: 0.3441
Epoch 10/100
363/363 [==============================] - 0s 809us/step - los

Beacuse train step is automatically stopped when the model doesn't imporve anymore, we can set number of epochs much.

### - Hyperparameters ###

Flexibility of neural network can be a disadvantage. we can control number of layers, number of neurons, activation funcion and weight initialization etc. Then how we know what combination of hyperparameters is best for given problem? One way is to try a lot of combination of hyperparameters and check what model has best score in validation set. We can search hyperparameter space using **GridSearchCV** and **RandomizedSearchCV**. To do this work, we have to change keras model to be seen like sklearn etimatior.


In [19]:
def keras_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for i in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

This function makes simple Sequential model for simple regression and compiles models for SGD SGD optimizer whcih uses designated learning rate.

In [20]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(keras_model)

**KerasRegressor** object is a simple wrapper that covers keras made by keras_model. Since we didn't set any hyperparameter it will use basic hyperparameter. Now we can use it like sklearn regression estimator.

In [21]:
keras_reg.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), 
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])
mse_test = keras_reg.score(X_test, y_test)

Epoch 1/100
363/363 [==============================] - 0s 934us/step - loss: 1.3910 - val_loss: 0.6606
Epoch 2/100
363/363 [==============================] - 0s 666us/step - loss: 0.6276 - val_loss: 0.5873
Epoch 3/100
363/363 [==============================] - 0s 689us/step - loss: 0.5704 - val_loss: 0.5505
Epoch 4/100
363/363 [==============================] - 0s 697us/step - loss: 0.5385 - val_loss: 0.5250
Epoch 5/100
363/363 [==============================] - 0s 729us/step - loss: 0.5171 - val_loss: 0.5089
Epoch 6/100
363/363 [==============================] - 0s 694us/step - loss: 0.5061 - val_loss: 0.5194
Epoch 7/100
363/363 [==============================] - 0s 908us/step - loss: 0.5074 - val_loss: 0.4919
Epoch 8/100
363/363 [==============================] - 0s 733us/step - loss: 0.4854 - val_loss: 0.5049
Epoch 9/100
363/363 [==============================] - 0s 676us/step - loss: 0.4787 - val_loss: 0.5116
Epoch 10/100
363/363 [==============================] - 0s 713us/step - l

We have to train hundreds model and choose best model in validation set. Since there are lots of hyperparameters, random search is better than grid search.

In [27]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

params_dict = {
    "n_hidden" : [0, 1, 2, 3] ,
    "n_neurons" : np.arange(1,100) ,
    "learning_rate" : reciprocal(3e-4, 3e-2)
}

rnd = RandomizedSearchCV(keras_reg, params_dict, n_iter=10, cv=3)
rnd.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid),
       callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
242/242 [==============================] - 0s 1ms/step - loss: 4.3114 - val_loss: 2.4800
Epoch 2/100
242/242 [==============================] - 0s 948us/step - loss: 1.6659 - val_loss: 1.1879
Epoch 3/100
242/242 [==============================] - 0s 1ms/step - loss: 1.0215 - val_loss: 0.9241
Epoch 4/100
242/242 [==============================] - 0s 968us/step - loss: 0.8551 - val_loss: 0.8153
Epoch 5/100
242/242 [==============================] - 0s 996us/step - loss: 0.7711 - val_loss: 0.7501
Epoch 6/100
242/242 [==============================] - 0s 927us/step - loss: 0.7193 - val_loss: 0.7080
Epoch 7/100
242/242 [==============================] - 0s 949us/step - loss: 0.6844 - val_loss: 0.6786
Epoch 8/100
242/242 [==============================] - 0s 994us/step - loss: 0.6594 - val_loss: 0.6572
Epoch 9/100
242/242 [==============================] - 0s 954us/step - loss: 0.6400 - val_loss: 0.6405
Epoch 10/100
242/242 [==============================] - 0s 980us/step - loss:

RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x000001CB5789EF88>, as the constructor either does not set or modifies parameter learning_rate

In [28]:
rnd.best_params_

{'learning_rate': 0.007210755428272255, 'n_hidden': 3, 'n_neurons': 16}

In [29]:
rnd.best_score_

-0.32181602716445923